# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import numpy as np
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism_detector_model'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-west-1-853058461749/plagiarism_detector_model


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism_detector_model/test.csv
plagiarism_detector_model/train.csv
sagemaker-pytorch-2020-08-20-06-37-59-754/source/sourcedir.tar.gz
sagemaker-pytorch-2020-08-20-06-39-27-892/source/sourcedir.tar.gz
sagemaker-pytorch-2020-08-20-06-51-07-459/debug-output/training_job_end.ts
sagemaker-pytorch-2020-08-20-06-51-07-459/output/model.tar.gz
sagemaker-pytorch-2020-08-20-06-51-07-459/source/sourcedir.tar.gz
sagemaker/sentiment_rnn/train.csv
sagemaker/sentiment_rnn/word_dict.pkl
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [5]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib
from sklearn.svm import SVC
# TODO: Import any additional libraries you need to define a model


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")

    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")

    return model


# TODO: Complete the main code
if __name__ == '__main__':

    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job

    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
    # Do not need to chang

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

### Helper function for evaluation

In [39]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True, model=''):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    if model == 'LinearLearner':
        test_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in predictor.predict(test_features)])
    elif model == 'PyTorchModel':
        test_preds = np.squeeze(np.round(predictor.predict(test_features))) # rounding and squeezing array
    else:
        test_preds = test_labels
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

---
# Model A: Built-in Classification Algorithbm - LinearLearner

## EXERCISE: I. Create train and test sets

In [13]:
import os

# read in train data and test data which are stored locally in data_dir and named train.csv, test.csv. 
train_data = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None)
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# The labels and features are extracted from the .csv file.
train_y = train_data.iloc[:,0]
train_x = train_data.iloc[:,1:]

test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

# convert features and labels to numpy array's float values
train_x_np = train_x.astype('float32').to_numpy()
train_y_np = train_y.astype('float32').to_numpy()

test_x_np = test_x.astype('float32').to_numpy()
test_y_np = test_y.astype('float32').to_numpy()

## EXERCISE: II. Create a LinearLearner Estimator

In [14]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner estimator
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)

# format the data as a RecordSet for training
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

## EXERCISE: III. Train the estimator

In [15]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-20 09:12:23 Starting - Starting the training job...
2020-08-20 09:12:26 Starting - Launching requested ML instances......
2020-08-20 09:13:38 Starting - Preparing the instances for training......
2020-08-20 09:14:42 Downloading - Downloading input data...
2020-08-20 09:15:09 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[08/20/2020 09:15:31 INFO 140186199975744] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'in

## EXERCISE: IV. Deploy the trained model

In [16]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!CPU times: user 324 ms, sys: 16 ms, total: 340 ms
Wall time: 7min 32s


## EXERCISE: V. Determine the accuracy of model

Use the deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [19]:
# First: generate predicted, class labels
result = linear_predictor.predict(test_x_np)
test_y_linear_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in result])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_linear_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [20]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
linear_accuracy = accuracy_score(test_y, test_y_linear_preds)

## print out the array of predicted and true labels, as well as the accuracy
print('\nPredicted class labels: ', test_y_linear_preds)
print('\nTrue class labels: ', test_y.values)
print('\nAccuracy: ', linear_accuracy)


Predicted class labels:  [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels:  [1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

Accuracy:  1.0


In [26]:
# get metrics for custom predictor
linear_predictor_metrics = evaluate(linear_predictor, test_x_np, test_y_np, True, 'LinearLearner')

predictions  0.0  1.0
actuals              
0.0           10    0
1.0            0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000




## EXERCISE: VI. Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [36]:
linear_predictor.delete_endpoint()

---
# Model B: Custom PyTorch Neural Network Classifier

## EXERCISE: I. Complete a training script 

In [27]:
!pygmentize source_pytorch/model.py

# torch imports
import torch.nn.functional as F
import torch.nn as nn


# TODO: Complete this classifier
class BinaryClassifier(nn.Module):
    """
    Define a neural network that performs binary classification.
    The network should accept your number of features as input, and produce
    a single sigmoid value, that can be rounded to a label: 0 or 1, as output.

    Notes on training:
    To train a binary classifier in PyTorch, use BCELoss.
    BCELoss is binary cross entropy loss, documentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss
    """

    # Define the init function, the input params are required (for loading code in train.py to work)
    def __init__(self, input_features, hidden_dim, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: help

In [28]:
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier


def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load

## EXERCISE: II. Create a PyTorch estimator

In [45]:
# import PyTorch
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch',
                    role=role,
                    framework_version='1.0',                    
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': 3,
                        'hidden_dim': 30,
                        'output_dim': 1,
                        'epochs': 90
                    })

## EXERCISE: III. Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [46]:
%%time

# Train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-20 10:38:08 Starting - Starting the training job...
2020-08-20 10:38:12 Starting - Launching requested ML instances.........
2020-08-20 10:39:40 Starting - Preparing the instances for training...
2020-08-20 10:40:36 Downloading - Downloading input data
2020-08-20 10:40:36 Training - Downloading the training image...
2020-08-20 10:41:06 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-20 10:40:56,791 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-20 10:40:56,794 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-20 10:40:56,806 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-20 10:40:59,821 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-20 10:41:00,086 sagemaker-containers INFO     Module train does 

## EXERCISE: IV. Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [33]:
!pygmentize source_pytorch/predict.py

# import libraries
import os
import numpy as np
import torch
from six import BytesIO

# import model from model.py, by name
from model import BinaryClassifier

# default content type is numpy array
NP_CONTENT_TYPE = 'application/x-npy'


# Provided model load function
def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the store model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with op

In [47]:
%%time
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data and point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.0',
                     entry_point='predict.py',
                     source_dir='source_pytorch')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 1.07 s


In [48]:
%%time
# deploy and create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!CPU times: user 381 ms, sys: 21.1 ms, total: 402 ms
Wall time: 6min 33s


In [44]:
predictor.delete_endpoint()

## EXERCISE: V. Determine the accuracy of model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [49]:
# First: generate predicted, class labels
test_y_preds = np.squeeze(np.round(predictor.predict(test_x)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [50]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

## print out the array of predicted and true labels, as well as the accuracy
print('\nPredicted class labels: ', test_y_preds)
print('\nTrue class labels: ', test_y.values)
print('\nAccuracy: ', accuracy)


Predicted class labels:  [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels:  [1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

Accuracy:  1.0


In [51]:
# get metrics for custom predictor
metrics = evaluate(predictor, test_x, test_y, True, 'PyTorchModel')

predictions  0.0  1.0
actuals              
0             10    0
1              0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000



### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**:

Model A used a built-in classification algorithm LinearLearner. Model B used a custom Pytorch neural network classifier.

Both Model A and Model B have 100% accuracy. There are no false positives and false negatives. According to Paul Clough research paper, containment and longest common subsequence are a good measures of the similarity between two paragraphs of text. Choosing good features plays an important role in improving the accuracy of the model.

```
predictions  0.0   1.0
actuals              
0.0           10    0
1.0            0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000
```

### Question 2: How did you decide on the type of model to use? 

**Answer:**
Since the goal of the model is simplified to determine whether the answer text is plagiarized or not, the built-in LinearLearner algorithm will be a good start. It turned out that the accuracy is 100%.

Then, I created a Pytorch classifier to test whether the higher accuracy of Model A is caused by the smaller data set. The BinaryClassifier of Model B includes linear layers, Relu as activation on hidden layer, regularization layer, and sigmoid layer to produce single value output. BCELoss() as loss function and Adam optimizer to train the model.

First try, the hyperparameters are set to `input_features`: 3, `hidden_dim`: 20, `output_dim`: 1, `epochs`: 80. Accuracy is only 72%.
```
predictions  0.0  1.0
actuals              
0              3    7
1              0   15

Recall:     1.000
Precision:  0.682
Accuracy:   0.720
```
Second try,  the hyperparameters are set to `input_features`: 3, `hidden_dim`: 30, `output_dim`: 1, `epochs`: 90. Accuracy reach 100%.
```
predictions  0.0  1.0
actuals              
0             10    0
1              0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000
```

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [ ]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()


In [52]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [53]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted sagemaker-pytorch-2020-08-20-10-42-43-273


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [54]:
# deleting bucket
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'E8954B45559D043E',
   'HostId': 'mw0iEXO1TLMWlFqRtpu+amebu1TW+g7HgHEK5kDddb1x7F3cJ6aCfISHcTfpV55oMV9pKztWIN0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'mw0iEXO1TLMWlFqRtpu+amebu1TW+g7HgHEK5kDddb1x7F3cJ6aCfISHcTfpV55oMV9pKztWIN0=',
    'x-amz-request-id': 'E8954B45559D043E',
    'date': 'Thu, 20 Aug 2020 10:52:43 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-record-sets/LinearLearner-2020-08-20-09-12-07-434/matrix_0.pbr'},
   {'Key': 'sagemaker-pytorch-2020-08-20-10-38-07-773/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-08-20-09-12-07-434/.amazon.manifest'},
   {'Key': 'plagiarism_detector_model/sagemaker-pytorch-2020-08-20-10-38-07-773/output/model.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-08-20-09-59-12-684/source/sourcedir.tar.gz'},
   {'Key': 'pl

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!